In [3]:
import os
import sys
import cv2

In [4]:
# ==================== CONFIGURAÇÃO ==================== #
input_folder = "/home/gotoxico/projetoBD2/Videos"                # Caminho da pasta com vídeos
output_frames_folder = "/home/gotoxico/projetoBD2/Frames"       # Caminho da pasta para salvar os frames
save_video = False                         # True = salva vídeos redimensionados também
save_every_nth_frame = 1                  # Salvar 1 a cada N frames
image_quality = 90                        # Qualidade JPEG (0–100)
# ====================================================== #

In [5]:
# extensões de vídeo comuns (maiúsculas/minúsculas)
VIDEO_EXTS = {".mp4", ".avi", ".mov", ".mkv", ".flv", ".wmv", ".webm", ".mpg", ".mpeg"}


def find_video_files(input_dir):
    files = []
    for entry in os.listdir(input_dir):
        path = os.path.join(input_dir, entry)
        if os.path.isfile(path):
            _, ext = os.path.splitext(entry)
            if ext.lower() in VIDEO_EXTS:
                files.append(path)
    return sorted(files)


def get_video_resolution(path):
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        raise RuntimeError(f"Não foi possível abrir o vídeo: {path}")
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    return width, height


def choose_smallest_resolution_by_area(video_paths):
    """Retorna (width, height) do menor vídeo por área (w*h)."""
    best = None
    for p in video_paths:
        w, h = get_video_resolution(p)
        area = w * h
        if best is None or area < best[0]:
            best = (area, w, h, p)
    if best is None:
        raise RuntimeError("Nenhum vídeo encontrado para determinar resolução.")
    _, w, h, path = best
    print(f"Menor resolução detectada: {w}x{h} (arquivo: {os.path.basename(path)})")
    return w, h


def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def process_videos(video_paths, out_frames_root, target_size, save_resized_video=False, out_video_root=None, save_every_nth_frame=1, image_quality=90):
    """
    - video_paths: lista de caminhos de vídeo
    - out_frames_root: pasta raiz onde serão criadas subpastas com frames
    - target_size: (width, height) alvo para redimensionamento
    - save_resized_video: se True, salva também arquivo de vídeo redimensionado
    - out_video_root: pasta para videos redimensionados (se save_resized_video True)
    - save_every_nth_frame: salva apenas 1 a cada N frames (use 1 para todos)
    - image_quality: JPEG quality 0-100
    """
    target_w, target_h = target_size
    ensure_dir(out_frames_root)
    if save_resized_video and out_video_root:
        ensure_dir(out_video_root)

    for video_path in video_paths:
        vid_name = os.path.splitext(os.path.basename(video_path))[0]
        out_folder = os.path.join(out_frames_root, vid_name)
        ensure_dir(out_folder)

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Aviso: não foi possível abrir {video_path}. Pulando.")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"Processando '{video_path}': {total_frames} frames, {fps:.2f} FPS -> salvando em {out_folder}")

        writer = None
        if save_resized_video and out_video_root:
            out_video_path = os.path.join(out_video_root, f"{vid_name}_resized.mp4")
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            # Nota: VideoWriter espera (width, height) da frame passada
            writer = cv2.VideoWriter(out_video_path, fourcc, fps if fps > 0 else 25.0, (target_w, target_h))

        frame_idx = 0
        saved_idx = 0
        encode_params = [int(cv2.IMWRITE_JPEG_QUALITY), int(image_quality)]

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # opcional: salvar apenas 1 a cada N frames
            if (frame_idx % save_every_nth_frame) == 0:
                # resize para target (note: cv2.resize usa (width, height) via dsize=(w,h))
                resized = cv2.resize(frame, (target_w, target_h), interpolation=cv2.INTER_AREA)
                # salvar imagem
                out_img_path = os.path.join(out_folder, f"{vid_name}_frame_{saved_idx:06d}.jpg")
                cv2.imwrite(out_img_path, resized, encode_params)
                saved_idx += 1

                # escrever vídeo redimensionado, se necessário
                if writer is not None:
                    writer.write(resized)

            frame_idx += 1

        cap.release()
        if writer is not None:
            writer.release()

        print(f"  -> {saved_idx} frames salvos em {out_folder}")


def main():
    videos = find_video_files(input_folder)
    if not videos:
        print("Nenhum arquivo de vídeo encontrado no diretório fornecido.")
        sys.exit(1)

    target_w, target_h = choose_smallest_resolution_by_area(videos)

    out_video_folder = None
    if save_video:
        out_video_folder = os.path.join(os.path.dirname(output_frames_folder), "resized_videos")
        ensure_dir(out_video_folder)

    process_videos(
        videos,
        output_frames_folder,
        (target_w, target_h),
        save_resized_video=save_video,
        out_video_root=out_video_folder,
        save_every_nth_frame=save_every_nth_frame,
        image_quality=image_quality
    )

    print("\nConcluído com sucesso!")


In [6]:
"""
resize_and_extract_frames.py

Descrição:
- Encontra todos os arquivos de vídeo numa pasta.
- Determina a menor resolução entre eles (por área = width * height).
- Redimensiona cada frame de cada vídeo para essa resolução e salva os frames em pastas separadas (uma pasta por vídeo).
- Opcional: salva também o vídeo redimensionado.

"""

if __name__ == "__main__":
    main()

Menor resolução detectada: 1280x720 (arquivo: VIRAT_S_000200_00_000100_000171.mp4)
Processando '/home/gotoxico/projetoBD2/Videos/VIRAT_S_000001.mp4': 20655 frames, 29.97 FPS -> salvando em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000001
  -> 20655 frames salvos em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000001
Processando '/home/gotoxico/projetoBD2/Videos/VIRAT_S_000102.mp4': 29580 frames, 29.97 FPS -> salvando em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000102
  -> 29580 frames salvos em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000102
Processando '/home/gotoxico/projetoBD2/Videos/VIRAT_S_000200_00_000100_000171.mp4': 2114 frames, 30.00 FPS -> salvando em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000200_00_000100_000171
  -> 2114 frames salvos em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000200_00_000100_000171
Processando '/home/gotoxico/projetoBD2/Videos/VIRAT_S_000200_04_000937_001443.mp4': 15164 frames, 30.00 FPS -> salvando em /home/gotoxico/projetoBD2/Frames/VIRAT_S_000200_04_0009